<a href="https://colab.research.google.com/github/samwondim/nlp-recommendation-image-classification/blob/main/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras


In [3]:
filename = 'https://github.com/lmassaron/datasets/releases/download/1.0/imdb_50k.feather'
reviews = pd.read_feather(filename)

print(reviews.review.sample(1).values[0])

The film was apparently spawned from an idea one of the writers had when he 'saw' one of his creations in a supermarket. The inhabitants of Royston Vasey head into 'our' world to persuade the writers not to stop writing about them and thus destroy their world.<br /><br />If that sounds a bit too serious, don't be put off. Within the first few minutes we get: Bernice (the vile female vicar) letting rip at an unfortunate penitent during confession; Chinnery (the vet who inadvertently destroys every animal he touches) attempting to collect semen from a giraffe; Mickey (thick beyond belief) being, ah, thick; and Tubbs (inbred sister-wife and local shopkeeper) being sweet as ever - but still disgusting.<br /><br />Some of the regular characters are missing, but a new idea by the Gents introduces some 16th-Century characters - and we have the Gents themselves in the action too. If you're new to The League of Gentlemen, this is an easy introduction and a lot of fun. If you're a long-standing 

In [ ]:
train = reviews.sample(3000, random_state=42)
sampled_idx = train.index
valid = (reviews[~reviews.index.isin(train.index)].sample(1000, random_state=42))
sampled_idx.append(valid.index)
test = reviews[~reviews.index.isin(sampled_idx)]


In [ ]:
# Instantiate a tokenizer
tokenizer = keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(train.review)

In [ ]:
import numpy as np
def tokenize_and_pad(tokenizer, df, maxlen):
  sequences = tokenizer.texts_to_sequences(df.review)
  pad = keras.preprocessing.sequence.pad_sequences
  padded_seqs = pad(sequences, maxlen)

  return padded_seqs, df.sentiment

words = { "review" : ["Hey my name is samuel", "Hey what's your name?"] }
w = pd.DataFrame(words.review)
print(w.head())

In [ ]:
X, y = tokenize_and_pad(tokenizer, train, maxlen=256)
Xv, yv = tokenize_and_pad(tokenizer, valid, maxlen=256)
Xt, yt = tokenize_and_pad(tokenizer, test, maxlen=256)

In [ ]:
# model = keras.models.Sequential()
tokenizer.index_word

In [ ]:
model = keras.models.Sequential()
voc = len(tokenizer.index_word) + 1
feats = 8
seq_len = 256

model.add(keras.layers.Embedding(voc, feats, input_length=seq_len))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

model.summary()

In [ ]:
history = model.fit(X, y, epochs=20, batch_size=16, validation_data=(Xv, yv))

In [ ]:
from sklearn.metrics import accuracy_score

predictions = (model.predict(Xt)>=0.5).astype(int)
test_accuracy = accuracy_score(yt, predictions)

print(f"Accuracy on test set: {test_accuracy}")